#### Evaluate fintuend model on customized data

In [61]:
import os,sys
sys.path.insert(0,'../libs')
import pandas as pd
import openai,json
from dotenv import load_dotenv
from tqdm import tqdm
from oai_fintune_utils import load_jsonl
from pydantic import BaseModel
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
env_path = '../../.env'
load_dotenv(dotenv_path=env_path)


True

In [62]:
data_folder='/root/data/home/xiong/data/Fund/CSR'
fintune_data_folder=os.path.join(data_folder,'Fintuning_data','Monetary')
train_data_path = os.path.join(fintune_data_folder,'train_mon_stance.jsonl')
eval_data_path = os.path.join(fintune_data_folder,'test_mon_stance.jsonl')

In [63]:
train_data= load_jsonl(train_data_path)
eval_data= load_jsonl(eval_data_path)

In [64]:
train_data[0]

{'messages': [{'role': 'system',
   'content': 'You are an experience macroeconomist from IMF. \nGiven a piece of text concerning a particular country in a given year expressing the views of IMF staff, complete the following two tasks. \n\nFirst, classify the country\'s recent or current monetary policy stance as described in the text into \n**restrictive/neutral/accommodative/unclear/irrelevant**; \nif it discusses monetary policy but the specific stance is not clear, assign unclear; \nif it does not discuss monetary policy, assign irrelevant. \n\nSecond, classify the IMF staff\'s recommended or planned near-future (next year) direction of change in monetary policy stance \nas described in the text into **tightening/tightening bias/no change/loosening bias/loosening/unclear/irrelevant**; \nif it discusses monetary policy stance but the direction of change is not clear, assign no change; \nif it does not discuss monetary policy stance, assign unclear (if it discusses monetary policy) \

In [65]:
train_data[0]['messages'][0]

{'role': 'system',
 'content': 'You are an experience macroeconomist from IMF. \nGiven a piece of text concerning a particular country in a given year expressing the views of IMF staff, complete the following two tasks. \n\nFirst, classify the country\'s recent or current monetary policy stance as described in the text into \n**restrictive/neutral/accommodative/unclear/irrelevant**; \nif it discusses monetary policy but the specific stance is not clear, assign unclear; \nif it does not discuss monetary policy, assign irrelevant. \n\nSecond, classify the IMF staff\'s recommended or planned near-future (next year) direction of change in monetary policy stance \nas described in the text into **tightening/tightening bias/no change/loosening bias/loosening/unclear/irrelevant**; \nif it discusses monetary policy stance but the direction of change is not clear, assign no change; \nif it does not discuss monetary policy stance, assign unclear (if it discusses monetary policy) \nor irrelevant (

#### Get Fintuned Model Id from OAI website 

In [66]:
class output(BaseModel):
    stance_current: str
    stance_future: str

- try one instance 

In [67]:
model_id = "ft:gpt-4o-mini-2024-07-18:personal::AKfGCd2g"

## create clinet and test api key
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)
## run one test 
messages=[
    train_data[0]['messages'][0],
    train_data[0]['messages'][1],
]
## just run one test, make sure the api works 
response = client.beta.chat.completions.parse(
    model=model_id, 
    messages=messages,
    temperature=0,
    response_format=output
)
print(response.choices[0].message.parsed)

stance_current='restrictive' stance_future='tightening bias'


#### run through all training datasets 

In [81]:
def get_oai_outputs(data):
    oai_outputs = []
    for td in tqdm(data):
        messages = td['messages'][:2]
        response = client.beta.chat.completions.parse(
            model=model_id, 
            messages=messages,
            temperature=0,
            response_format=output
        )
        oai_outputs.append(response.choices[0].message.parsed)
    return oai_outputs

In [82]:
train_oai_outputs = get_oai_outputs(train_data)
train_ground_truth = [eval(td['messages'][-1]['content'])['stance_current'] for td in train_data]
model_outputs = [td.stance_current for td in train_oai_outputs]


100%|██████████| 462/462 [06:24<00:00,  1.20it/s]


0.8311688311688312

In [85]:
print('training data accuracy: {}'.format(accuracy_score(train_ground_truth,model_outputs)))

training data accuracy: 0.8311688311688312


In [87]:
eval_oai_outputs = get_oai_outputs(eval_data)
train_ground_truth = [eval(td['messages'][-1]['content'])['stance_current'] for td in eval_data]
eval_model_outputs = [td.stance_current for td in eval_oai_outputs]

100%|██████████| 116/116 [01:25<00:00,  1.36it/s]


In [88]:
accuracy_score(train_ground_truth,eval_model_outputs)

0.7155172413793104